In [4]:
!curl -O "https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/datasets/spacex_launch_dash.csv"


  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  2476  100  2476    0     0   1855      0  0:00:01  0:00:01 --:--:--  1854


In [6]:
import pandas as pd

# Read the CSV file into a DataFrame
spacex_df = pd.read_csv("spacex_launch_dash.csv")

# Display the first 5 rows of the DataFrame
spacex_df.head()


,Unnamed: 0,Flight Number,Launch Site,class,Payload Mass (kg),Booster Version,Booster Version Category
0,0,1,CCAFS LC-40,0,0.0,F9 v1.0 B0003,v1.0
1,1,2,CCAFS LC-40,0,0.0,F9 v1.0 B0004,v1.0
2,2,3,CCAFS LC-40,0,525.0,F9 v1.0 B0005,v1.0
3,3,4,CCAFS LC-40,0,500.0,F9 v1.0 B0006,v1.0
4,4,5,CCAFS LC-40,0,677.0,F9 v1.0 B0007,v1.0


In [10]:
!pip install dash


  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.0/8.0 MB 4.5 MB/s eta 0:00:0000:0100:01
  Created wheel for stringcase: filename=stringcase-1.2.0-py3-none-any.whl size=3569 sha256=9aecb14f2927d02de52a44efdcb505fbc1991391603e049385857469e00ed482
  Stored in directory: /Users/nemethlaszlo/Library/Caches/pip/wheels/37/76/84/09143757ac9846675769f3649fc2aa936ec40255783a23ba52
Successfully built stringcase


In [23]:
# Import required libraries
import pandas as pd
import dash
from dash import dcc, html
from dash.dependencies import Input, Output
import plotly.express as px

# Read the SpaceX Launch data into a DataFrame
spacex_df = pd.read_csv("spacex_launch_dash.csv")
max_payload = spacex_df['Payload Mass (kg)'].max()
min_payload = spacex_df['Payload Mass (kg)'].min()

# Create a Dash application
app = dash.Dash(__name__)

# Create an app layout with a dropdown, pie chart, payload slider, and scatter chart
app.layout = html.Div(children=[
    html.H1("SpaceX Launch Records Dashboard",
            style={'textAlign': 'center', 'color': '#503D36', 'font-size': 40}),
    
    # TASK 1: Add a dropdown list to enable Launch Site selection
    dcc.Dropdown(
        id='site-dropdown',
        options=[
            {'label': 'All Sites', 'value': 'ALL'},
            {'label': 'CCAFS LC-40', 'value': 'CCAFS LC-40'},
            {'label': 'VAFB SLC-4E', 'value': 'VAFB SLC-4E'},
            {'label': 'KSC LC-39A', 'value': 'KSC LC-39A'},
            {'label': 'CCAFS SLC-40', 'value': 'CCAFS SLC-40'}
        ],
        value='ALL',
        placeholder="Select a Launch Site here",
        searchable=True
    ),
    
    html.Br(),
    
    # TASK 2: Add a pie chart to show total successful launches count
    # If a specific launch site is selected, show the Success vs. Failed counts for that site
    html.Div(dcc.Graph(id='success-pie-chart')),
    
    html.Br(),
    
    html.P("Payload range (Kg):"),
    
    # TASK 3: Add a slider to select payload range
    dcc.RangeSlider(
        id='payload-slider',
        min=0,
        max=10000,
        step=1000,
        marks={
            0: '0',
            2500: '2500',
            5000: '5000',
            7500: '7500',
            10000: '10000'
        },
        value=[min_payload, max_payload]
    ),
    
    html.Br(),
    
    # TASK 4: Add a scatter chart to show the correlation between payload and launch success
    html.Div(dcc.Graph(id='success-payload-scatter-chart'))
])

# TASK 2: Callback to update the pie chart based on the selected launch site
@app.callback(
    Output('success-pie-chart', 'figure'),
    [Input('site-dropdown', 'value')]
)
def update_pie_chart(selected_site):
    if selected_site == 'ALL':
        # Group data by Launch Site and sum the success count (assuming 'class' indicates success with 1)
        df_grouped = spacex_df.groupby('Launch Site', as_index=False)['class'].sum()
        fig = px.pie(
            df_grouped,
            names='Launch Site',
            values='class',
            title='Total Success Launches by Site'
        )
    else:
        # Filter the DataFrame for the selected site
        filtered_df = spacex_df[spacex_df['Launch Site'] == selected_site]
        # Create a pie chart showing the counts of success (class=1) vs. failure (class=0)
        fig = px.pie(
            filtered_df,
            names='class',
            title=f"Total Success Launches for site {selected_site}"
        )
    return fig

# TASK 4: Callback to update the scatter chart based on selected launch site and payload range
@app.callback(
    Output('success-payload-scatter-chart', 'figure'),
    [
        Input('site-dropdown', 'value'),
        Input('payload-slider', 'value')
    ]
)
def update_scatter_chart(selected_site, payload_range):
    low, high = payload_range
    # Filter data based on the selected payload range
    mask = (spacex_df['Payload Mass (kg)'] >= low) & (spacex_df['Payload Mass (kg)'] <= high)
    filtered_df = spacex_df[mask]
    
    if selected_site != 'ALL':
        # Further filter the DataFrame for the selected launch site if not 'ALL'
        filtered_df = filtered_df[filtered_df['Launch Site'] == selected_site]
    
    # Create a scatter chart showing the correlation between payload and launch success
    fig = px.scatter(
        filtered_df,
        x='Payload Mass (kg)',
        y='class',
        color='Booster Version Category',
        title='Correlation between Payload and Success'
    )
    return fig

# Run the app
if __name__ == '__main__':
    app.run(debug=True)


**1. Which site has the largest successful launches?**  
Based on the pie chart (using the "ALL SITES" option), **KSC LC-39A** shows the largest number of successful launches (e.g., around 41.7%).

**2. Which site has the highest launch success rate?**  
According to the data, **VAFB SLC-4E** often exhibits the highest success rate (up to 100%), but this should be confirmed by comparing the success/failure ratios for each site.

**3. Which payload range(s) has the highest launch success rate?**  
Analysis of the scatter chart indicates that the payload range around **2000–4000 kg** generally shows the highest launch success rate, as most launches in this range have a success (`class = 1`) outcome.

**4. Which payload range(s) has the lowest launch success rate?**  
Often, the payload range above **7500 kg** (or very low payloads) shows a higher number of failures (`class = 0`), indicating a lower success rate.

**5. Which F9 Booster version has the highest launch success rate?**  
Generally, the **B5 (Block 5)** or **FT (Full Thrust)** versions are associated with the highest success rates (nearly 100%), but it is advisable to verify this by observing the color-coded points in the scatter chart.
  
*Note: The specific numbers and percentages depend on the dataset provided in `spacex_launch_dash.csv`. Adjust your answers based on the actual data if needed.*
